url = https://sy-log.tistory.com/entry/%EC%85%80%EB%A0%88%EB%8B%88%EC%9B%80-%ED%81%AC%EB%A1%A4%EB%A7%81-%EC%98%88%EC%A0%9C-%EB%8B%A4%EB%82%98%EC%99%80-%EB%A6%AC%EB%B7%B0-%ED%81%AC%EB%A1%A4%EB%A7%81%EC%9D%84-%EC%9C%84%ED%95%9C-%EC%A7%80%EC%86%8D%EA%B0%80%EB%8A%A5%ED%95%9C-%EC%BD%94%EB%93%9C-%EB%A7%8C%EB%93%A4%EA%B8%B0-2?category=984185

# 2-1. 제품 리뷰 페이지 넘기며 크롤링

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import time
import math
import pandas as pd
import pickle


In [3]:
df_products = pickle.load(open('danawa_crawling_airpurifier_result_240607.pk1','rb'))
df_products.head()

,제품모델명,출시시기,리뷰평점,리뷰개수,헤드메시지,스펙,링크 url,가격정보_1,최저가_1,가격정보_2,최저가_2,가격정보_3,최저가_3
0,LG전자 퓨리케어 360˚ 플러스 AS303DWFA,2023.01.,4.8,152,이제 플러스 모델에도 UP가전으로 업그레이드 가능!,공기청정기 / 30.3평(100㎡) / 소비전력: 70W / 버튼잠금 / 풍량조절 ...,https://prod.danawa.com/info/?pcode=18775037&c...,일반구매,"736,090원",무빙휠 포함,"856,970원",,
1,삼성전자 블루스카이 3100 AX033B310GGD,2022.03.,4.7,"1,065",,공기청정기 / 10평(33㎡) / 소비전력: 26W / 버튼잠금 / 풍량조절 / 스...,https://prod.danawa.com/info/?pcode=16536392&c...,1개,"153,000원",2개,"382,700원",추가필터 포함,"269,000원"
2,LG전자 오브제컬렉션 퓨리케어 360˚ UV살균 AS354NS3A,2024.02.,4.9,69,LG전자 24년형 PA인증받은 UV살균 공기청정기 출시!,공기청정기 / 34평(114㎡) / 소비전력: 80W / 버튼잠금 / 풍량조절 / ...,https://prod.danawa.com/info/?pcode=34738715&c...,일반구매,"1,311,990원",무빙휠 포함,"1,423,760원",,
3,LG전자 퓨리케어 360˚ Hit AS153HWWC,2023.03.,4.8,678,퓨리케어360˚ 1단 모델 대비 16% 슬림해진 에어라운드 디자인 적용,공기청정기 / 15평(50㎡) / 소비전력: 34W / 버튼잠금 / 풍량조절 / 스...,https://prod.danawa.com/info/?pcode=19312541&c...,,"311,380원",,,,
4,삼성전자 블루스카이 5500 AX060CG500GGD,2023.01.,4.6,91,실내 공기질까지 학습해 맞춰주는 AI 청정기능 적용,공기청정기 / 18평(60㎡) / 소비전력: 60W / 버튼잠금 / 풍량조절 / 이...,https://prod.danawa.com/info/?pcode=18852014&c...,일반구매,"247,690원",추가필터 포함,"419,000원",,


In [3]:
df_products.dtypes

제품모델명     object
출시시기      object
리뷰평점      object
리뷰개수      object
헤드메시지     object
스펙        object
링크 url    object
가격정보_1    object
최저가_1     object
가격정보_2    object
최저가_2     object
가격정보_3    object
최저가_3     object
dtype: object

# 2-1. 페이지 넘기면서 리뷰 크롤링

상세페이지 url 접속 -> 상품 리뷰 이동 -> 리뷰 페이지 넘기면서 -> 페이지 소스 가져오기


In [4]:
#크롬 드라이버 실행

my_options = webdriver.ChromeOptions()
my_options.add_argument("--window-size=1920x1080")
my_options.add_argument("disable-gpu")

driver = webdriver.Chrome(options=my_options)

In [5]:
#(1) 첫번째 제품 상세페이지 이동

url = df_products['링크 url'][0]
driver.get(url)

In [6]:
#(2) '상품리뷰' 클릭
driver.find_element(By.XPATH, '//*[@id="productReviewArea"]/div[1]/a/h4').click()

In [7]:
#리뷰 한 페이지에 리뷰 몇 개인지 알아보기
num_review = len(driver.find_elements(By.XPATH, '//*[@id="danawa-prodBlog-companyReview-content-list"]/ul/li'))
num_review

10

In [8]:
#리뷰가 총 몇페이지까지 있는지 계산
num_page = int((int(df_products['리뷰개수'][0])/num_review)+1)
num_page

16

In [9]:
#페이지별 소스 저장할 빈 딕셔너리 생성
soup_dict = {}

page = 1

#page에 1씩 더하면서, 전체 페이지 수와 같아지기 전까지 반복
while page < num_page:
    
    #페이지별 리뷰 전부 가져오기
    soup = BeautifulSoup(driver.page_source)
    reviews = soup.select('ul.rvw_list li.danawa-prodBlog-companyReview-clazz-more')
    time.sleep(1)
    
    #페이지별 리뷰 딕셔너리에 저장
    soup_dict[page] = reviews
    print(page, '페이지 완료')
    
    page += 1
    
    
    #11,21,31 번째 페이지인 경우 [다음 페이지] 버튼 클릭
    if page % 10 == 1:
        driver.find_element(By.XPATH, '//*[@id="danawa-prodBlog-companyReview-content-list"]/div/div/a').click()
        
    #나머지 경우는 page에 해당하는 페이지 클릭
    else:
        #페이지 넘기는 부분 경로 통째로 가져와
        page_num = driver.find_element(By.XPATH, '//*[@id="danawa-prodBlog-companyReview-content-list"]/div/div')
        
        #다음 페이지 링크 클릭
        page_num.find_element(By.LINK_TEXT, str(page)).click()
        
    time.sleep(2)
    

#마지막 페이지 리뷰 가져오고 마무리
soup = BeautifulSoup(driver.page_source)
reviews = soup.select('ul.rvw_list li.danawa-prodBlog-companyReview-clazz-more')
soup_dict[page] = reviews

print(page, '페이지 완료')

print('<< 테스트 크롤링 완료 >>')

1 페이지 완료
2 페이지 완료
3 페이지 완료
4 페이지 완료
5 페이지 완료
6 페이지 완료
7 페이지 완료
8 페이지 완료
9 페이지 완료
10 페이지 완료
11 페이지 완료
12 페이지 완료
13 페이지 완료
14 페이지 완료
15 페이지 완료
16 페이지 완료
<< 테스트 크롤링 완료 >>


# 2-2 저장해둔 페이지 소스에서 쓸모있는 정보만 가져와 DataFrame에 

In [10]:
#별점, 구매처, 리뷰작성일, 리뷰내용 수집하기

In [12]:
#모델이름, 별점, 구매처, 작성일, 리뷰내용
name_list, star_list, mall_list, date_list, review_list = [], [], [], [], []

### 딕셔너리에 저장해둔 페이지 하나씩 돌리기 ###
for key in range(len(soup_dict)):
    reviews = soup_dict[key+1]
    
    #리뷰마다 돌리면서 모델이름 등등 가져오기
    for i in range(len(reviews)):
        
        #모델 이름
        try:
            name_list.append(str(df_products['제품모델명'][0]))
        except:
            name_list.append('')
            
        #별점
        try:
            star_list.append(int
                            (reviews[i].select_one('span,star_mask').text.strip().replace('점',''))/20)
        except:
            star_list.append('')
            
        #구매처
        try:
            mall_list.append(str(reviews[i].select_one('span.mall').text.strip()))
        except:
            mall_list.append('')
        
        #작성일
        try:
            date_list.append(str(reviews[i].select_one('span.date').text.strip()))
        except:
            date_list.append('')            

        #리뷰내용
        try:
            review_list.append(str(reviews[i].select_one('div.atc').text.strip()))
        except:
            review_list.append('')
            

### 크롤링 결과 저장 ###
#데이터 프레임 생성
df_result = pd.DataFrame({
    '제품모델명':name_list, '구매처':mall_list, '작성일': date_list, '별점': star_list, '리뷰':review_list
}); df_result

,제품모델명,구매처,작성일,별점,리뷰
0,LG전자 퓨리케어 360˚ 플러스 AS303DWFA,엘지이닷컴,2024.04.30.,5.0,오랫동안 갖고 싶었던 제품입니다. 일단 디자인 너무 예쁘고 헤드 부분이 공기질에 따...
1,LG전자 퓨리케어 360˚ 플러스 AS303DWFA,엘지이닷컴,2024.04.24.,5.0,퓨리케어 정말좋네요~ 많이들 사용하는이유를\n알겠더라구요.소음도 거의없고~ 알아서 ...
2,LG전자 퓨리케어 360˚ 플러스 AS303DWFA,엘지이닷컴,2024.06.08.,5.0,집안 미세먼지 걱정에 구입을 했습니다. 구입후 미세먼지 뿐만 아니라 냄새까지 잡아주...
3,LG전자 퓨리케어 360˚ 플러스 AS303DWFA,엘지이닷컴,2024.04.15.,5.0,자동 공기청정 모드로 해놓으면 알아서 공기청정 해줍니다.\n공기가 상쾌해지는 기분이에요
4,LG전자 퓨리케어 360˚ 플러스 AS303DWFA,엘지이닷컴,2024.06.03.,5.0,강아지가 있고 미세먼지 등 요즘 너무 걱정이여서 구매한 제품이에요~~ 구독으로 했으...
...,...,...,...,...,...
146,LG전자 퓨리케어 360˚ 플러스 AS303DWFA,옥션,2024.01.13.,5.0,좋아요
147,LG전자 퓨리케어 360˚ 플러스 AS303DWFA,옥션,2024.01.05.,5.0,받은분들이 너무 만족해하십니다잘 쓸게요~
148,LG전자 퓨리케어 360˚ 플러스 AS303DWFA,11번가,2024.02.13.,5.0,잘 도착했습니다. 사용설명서 보면서 필터 비닐 제거하고 돌려보니 시원하게 잘 돌아갑니다.
149,LG전자 퓨리케어 360˚ 플러스 AS303DWFA,전자랜드 프라이스킹,2024.05.03.,5.0,좋아요 좋아요 좋아요 좋아요 좋아요 좋아요
